https://github.com/songys/Chatbot_data

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools
import math
from torch.jit import script, trace
from torch import optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
corpus_name = 'chatData'
corpus = os.path.join(r'C:\Users\admin\jupyter\pytorch\chatbot', corpus_name)

def printLines(file, n=10):
    with open(file, 'r', encoding='utf-8') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)
        
printLines(os.path.join(corpus, 'ChatbotData.csv'))

Q,A,label

12시 땡!,하루가 또 가네요.,0

1지망 학교 떨어졌어,위로해 드립니다.,0

3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0

3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0

PPL 심하네,눈살이 찌푸려지죠.,0

SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0

SD카드 안돼,다시 새로 사는 게 마음 편해요.,0

SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0

SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0



In [3]:
def loadLines(fileName, fields):
    lines = []
    with open(fileName, 'r', encoding='utf-8') as f:
        next(f)
        for line in f:
            values = line.split(',')
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lineObj['text'] = lineObj['Question'] + '\t' + lineObj['Answer']
            lines.append(lineObj['text'])
    return lines
     

lines = []
LINES_FIELDS = ['Question', 'Answer', 'text']
lines = loadLines(os.path.join(corpus, 'ChatbotData.csv'), LINES_FIELDS)
print(lines[:5])
print(len(lines))

['12시 땡!\t하루가 또 가네요.', '1지망 학교 떨어졌어\t위로해 드립니다.', '3박4일 놀러가고 싶다\t여행은 언제나 좋죠.', '3박4일 정도 놀러가고 싶다\t여행은 언제나 좋죠.', 'PPL 심하네\t눈살이 찌푸려지죠.']
11823


In [4]:
datafile = os.path.join(corpus, "formatted_lines.txt")

delimiter = '\t'
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in lines:
        writer.writerow([pair])

# 몇 줄을 예제 삼아 출력해 봅니다
print("\nSample lines from file:")
printLines(datafile)


Writing newly formatted file...

Sample lines from file:
"12시 땡!	하루가 또 가네요."

"1지망 학교 떨어졌어	위로해 드립니다."

"3박4일 놀러가고 싶다	여행은 언제나 좋죠."

"3박4일 정도 놀러가고 싶다	여행은 언제나 좋죠."

"PPL 심하네	눈살이 찌푸려지죠."

"SD카드 망가졌어	다시 새로 사는 게 마음 편해요."

"SD카드 안돼	다시 새로 사는 게 마음 편해요."

"SNS 맞팔 왜 안하지ㅠㅠ	잘 모르고 있을 수도 있어요."

"SNS 시간낭비인 거 아는데 매일 하는 중	시간을 정하고 해보세요."

"SNS 시간낭비인데 자꾸 보게됨	시간을 정하고 해보세요."



In [5]:
PAD_token = 0  # 패딩 토큰
SOS_token = 1  # 시작 토큰
EOS_token = 2  # 끝 토큰

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False 
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # SOS, EOS, PAD를 센 것

    # 단어 분리
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    # 단어 집합 생성
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # 등장 횟수가 기준 이하인 단어를 정리합니다
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)))

        # 사전을 다시 초기화힙니다
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 

        for word in keep_words:
            self.addWord(word)

In [6]:
MAX_LENGTH = 10  

# 질의/응답 쌍을 읽어서 voc 객체를 반환합니다
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # 파일을 읽고, 쪼개어 lines에 저장합니다
    lines = open(datafile, encoding='utf-8').read().strip().split('\n')
    pairs = [l.split('\t') for l in lines]
    pairs = sum(pairs, []) # 2차원에서 1차원으로
    voc = Voc(corpus_name)
    return voc, pairs

def filterPair(p):
    # EOS 토큰을 위해 입력 시퀀스의 마지막 단어를 보존해야 합니다
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# 조건식 filterPair에 따라 pairs를 필터링합니다
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name) # voc : 단어집합, pairs : 질문 쌍
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)

for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 23646 sentence pairs


IndexError: string index out of range

In [ ]:

# 문장의 쌍 'p'에 포함된 두 문장이 모두 MAX_LENGTH라는 기준보다 짧은지를 반환합니다
def filterPair(p):
    # EOS 토큰을 위해 입력 시퀀스의 마지막 단어를 보존해야 합니다
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# 조건식 filterPair에 따라 pairs를 필터링합니다
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# 앞에서 정의한 함수를 이용하여 만든 voc 객체와 리스트 pairs를 반환합니다
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name) # voc : 단어집합, pairs : 질문 쌍
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# voc와 pairs를 읽고 재구성합니다
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)

print("\npairs:")
for pair in pairs[:10]:
    print(pair)